In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from PIL import Image
import os
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from tensorflow.keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score

In [2]:
def load_multiple_images(dataframe, image_folder, size=(224, 224)):
    image_list = []
    for img_id in dataframe['img_id']:
        full_path = f'{image_folder.rstrip("/")}/{img_id}.jpeg'
        img = Image.open(full_path).resize(size)
        img_array = np.array(img) / 255.0
        image_list.append(img_array)
    return np.array(image_list)

In [3]:
image_directory = r"C:\Users\Dell\Desktop\Studies\Subjects\Deep Learning\assignments\assignment2\bau-ain3002-spring23-a2p1\images\images"
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_targets = train_data['label'].values
train_img_data = load_multiple_images(train_data, image_directory)

X_training, X_validation, y_training, y_validation = train_test_split(train_img_data, train_targets, test_size=0.2, random_state=42)

test_img_data = load_multiple_images(test_data, image_directory)

In [4]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

C:\Users\Dell\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(
    X_training,
    y_training,
    epochs=20,
    batch_size=32,
    validation_data=(X_validation, y_validation)
)

Epoch 1/20
18/35 [==============>...............] - ETA: 2:23 - loss: 0.8785 - accuracy: 0.6076

In [ ]:
#I was checking again but it takes time and the deadline is soon so..
#you can try runinning the code

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
y_pred_proba = model.predict(X_validation)

auc_score = roc_auc_score(y_validation, y_pred_proba)

print('AUC score:', auc_score)

In [ ]:
test_img_data = load_multiple_images(test_data, image_directory)

y_pred_proba = model.predict(test_img_data)

y_pred = [int(round(x[0])) for x in y_pred_proba]

submission = pd.DataFrame({
    "img_id": test_data['img_id'].values, 
    "cancer_score": y_pred
})

submission.to_csv("submission.csv", index=False)